In [3]:
from bs4 import BeautifulSoup
import urllib.parse
import urllib.request
import re
import string
url = "https://www.allrecipes.com/recipe/45933/creamy-cajun-shrimp-pasta"
req = urllib.request.Request(url)
req.add_header('Cookie', 'euConsent=true')

html_content = urllib.request.urlopen(req).read()
soup = BeautifulSoup(html_content, 'html.parser')
direction_pattern = re.compile(r'<span class="recipe-directions__list--item">(?:(?!<span class="recipe-directions__list--item">|<\/span>)[\s\S])*<\/span>')

#ingredients = re.findall(ingredient_pattern, 'itemprop="recipeIngredient">1/4 cup chicken broth</span>')
direction = re.findall(direction_pattern, str(soup))

In [5]:
data = {
    "Title": 0,
    "Ingredients":{
        "Ingredient name": 0,
        "Quantity": 0,
        "Measurement": 0,
        "Descriptor": 0,
        "Preparation0": 0
    },
    "Tools": [],
    "Methods": {
        "Primary": [],
        "Other": []
    },
    "Steps":{
        1: {"Ingredients":[], "Tools": [], "Methods":[]},
    }
}

In [4]:
direction

['<span class="recipe-directions__list--item">Bring a large pot of lightly salted water to a boil. Add pasta and cook for 4 minutes or until al dente; drain.\n                            </span>',
 '<span class="recipe-directions__list--item">Melt butter in a large heavy skillet over medium heat. Saute shrimp for 1 minute on each side. Stir in garlic, and cook for 1 minute. Remove shrimp with a slotted spoon; set aside. Stir in flour and Cajun seasoning. Cook, stirring for 5 minutes. Gradually whisk in milk, then cook until thickened. Remove from heat, and season with salt and lemon juice. Return shrimp to sauce, and spoon over cooked pasta.\n                            </span>']

In [80]:
from cooktool import COOK_TOOL
from methods import METHODS

In [64]:
recipe = dict()

In [78]:
direction_list = []
rough_direction_list = []
for item in direction:
    item_tmp1 = re.split("<span class=\"recipe-directions__list--item\">", item)
    item_tmp2 = re.split("\n", item_tmp1[1])
    item_tmp3_list = re.split(r'\.|;',item_tmp2[0].lower())
    #rough_direction_list.append(item_tmp2[0].translate(str.maketrans('', '', string.punctuation)).lower())
    for subitem in item_tmp3_list:
        if subitem != '':
            direction_list.append(subitem.translate(str.maketrans('', '', string.punctuation)))

In [79]:
direction_list

['bring a large pot of lightly salted water to a boil',
 ' add pasta and cook for 4 minutes or until al dente',
 ' drain',
 'melt butter in a large heavy skillet over medium heat',
 ' saute shrimp for 1 minute on each side',
 ' stir in garlic and cook for 1 minute',
 ' remove shrimp with a slotted spoon',
 ' set aside',
 ' stir in flour and cajun seasoning',
 ' cook stirring for 5 minutes',
 ' gradually whisk in milk then cook until thickened',
 ' remove from heat and season with salt and lemon juice',
 ' return shrimp to sauce and spoon over cooked pasta']

In [115]:
TIME_WORD = {
    'hours': [re.compile(r'(about )?\d+ h(ou)?rs?($|\W)'), re.compile(r'\d+ to \d+ h(ou)?rs?($|\W)')],
    'minutes': [re.compile(r'(about )?\d+ min(ute)?s?($|\W)'), re.compile(r'\d+ to \d+ min(ute)?s?($|\W)')],
    'seconds': [re.compile(r'(about )?\d+ sec(ond)?s?($|\W)'), re.compile(r'\d+ to \d+ sec(ond)?s?($|\W)')]
}

In [129]:
recipe['methods'] = {}
recipe['methods']['primary'] = []
recipe['methods']['other'] = []
recipe['tools'] = []
recipe['steps'] = {}
count = 1
for item in direction_list:
    recipe['steps'][count] = dict()
    recipe['steps'][count]['methods'] = dict()
    recipe['steps'][count]['methods']['primary'] = []
    recipe['steps'][count]['methods']['other'] = []
    recipe['steps'][count]['tools'] = []
    recipe['steps'][count]['time'] = []
    
    for check_word in COOK_TOOL:
        if len(re.findall(COOK_TOOL[check_word], item))> 0:
            if check_word not in recipe['tools']:
                recipe['tools'].append(check_word)
            if check_word not in recipe['steps'][count]['tools']:
                recipe['steps'][count]['tools'].append(check_word)
            
    for check_word in METHODS['primary']:
        if check_word in item:
            if check_word not in recipe['methods']['primary']:
                recipe['methods']['primary'].append(check_word)
            if check_word not in recipe['steps'][count]['methods']['primary']:
                recipe['steps'][count]['methods']['primary'].append(check_word)
        elif check_word == 'sauté' and 'saute' in item:
            if check_word not in recipe['methods']['primary']:
                recipe['methods']['primary'].append(check_word)
            if check_word not in recipe['steps'][count]['methods']['primary']:
                recipe['steps'][count]['methods']['primary'].append(check_word)               
                
    for check_word in METHODS['other']:
        if check_word in item:
            if check_word not in recipe['methods']['other']:
                recipe['methods']['other'].append(check_word) 
            if check_word not in recipe['steps'][count]['methods']['other']:
                recipe['steps'][count]['methods']['other'].append(check_word)
    
    for check_type in TIME_WORD:
        for check_pattern in TIME_WORD[check_type]:
            search_res = re.search(check_pattern, item)
            if search_res:
                recipe['steps'][count]['time'].append(search_res.group(0))
    count += 1

In [130]:
recipe['steps']

{1: {'methods': {'primary': ['boil'], 'other': []},
  'tools': ['pot'],
  'time': []},
 2: {'methods': {'primary': [], 'other': []},
  'tools': [],
  'time': ['4 minutes ']},
 3: {'methods': {'primary': [], 'other': ['drain']}, 'tools': [], 'time': []},
 4: {'methods': {'primary': [], 'other': ['melt', 'heat']},
  'tools': ['skillet'],
  'time': []},
 5: {'methods': {'primary': ['sauté'], 'other': []},
  'tools': [],
  'time': ['1 minute ']},
 6: {'methods': {'primary': [], 'other': ['stir']},
  'tools': [],
  'time': ['1 minute']},
 7: {'methods': {'primary': [], 'other': []}, 'tools': ['spoon'], 'time': []},
 8: {'methods': {'primary': [], 'other': []}, 'tools': [], 'time': []},
 9: {'methods': {'primary': [], 'other': ['stir', 'season']},
  'tools': [],
  'time': []},
 10: {'methods': {'primary': [], 'other': ['stir']},
  'tools': [],
  'time': ['5 minutes']},
 11: {'methods': {'primary': [], 'other': ['whisk']},
  'tools': ['whisk'],
  'time': []},
 12: {'methods': {'primary': [], 